Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [190]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
from heapq import heappop, heappush, nsmallest
import numpy as np

In [191]:
PUZZLE_DIM = 3
RANDOMIZE_STEPS = 1000
action = namedtuple('Action', ['pos1', 'pos2'])

In [192]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def is_goal(state: np.ndarray) -> bool:
    """Check if the state matches the goal configuration."""
    goal = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    return np.array_equal(state, goal)

def difference_from_goal(state: np.ndarray) -> int:
    """Calculate the difference between the current state and the goal state."""
    goal = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    return np.sum(state != goal)

The Approach done is the A* search algorithm, where the heuristic function is the Manhattan distance

In [193]:
def manhattan_distance(state: np.ndarray) -> int:
    """Calculate the Manhattan distance of the current state to the goal state."""
    distance = 0
    goal_positions = {i: (i // PUZZLE_DIM, i % PUZZLE_DIM) for i in range(1, PUZZLE_DIM ** 2)}
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:
                goal_x, goal_y = goal_positions[value]
                distance += abs(x - goal_x) + abs(y - goal_y)
    return distance

def linear_conflict(state: np.ndarray) -> int:
    conflict = 0
    for i in range(PUZZLE_DIM):
        row = state[i, :]
        col = state[:, i]
        conflict += sum((x > y) for j, x in enumerate(row) for y in row[j + 1:])
        conflict += sum((x > y) for j, x in enumerate(col) for y in col[j + 1:])
    return conflict


In [194]:
def A_star(initial_state: np.ndarray):
    """Solve the puzzle using A* search with a priority queue (heapq)."""
    open_list = []
    heappush(open_list, (manhattan_distance(initial_state), initial_state.tobytes(), []))
    closed_set = set()
    closed_set.add(initial_state.tobytes())  # Use byte representation for consistency

    while open_list:
        # Pop the state with the lowest heuristic value
        heuristic, current_state_bytes, path = heappop(open_list)
        current_state = np.frombuffer(current_state_bytes, dtype=int).reshape(initial_state.shape)

        # Check if the goal state is reached
        if is_goal(current_state):
            return path, len(path), len(closed_set)

        # Generate successors
        for action in available_actions(current_state):
            next_state = do_action(current_state, action)
            next_state_bytes = next_state.tobytes()

            # Only process unvisited states
            if next_state_bytes not in closed_set:
                next_path = path + [action]
                priority = manhattan_distance(next_state) + 2*linear_conflict(next_state)+ len(next_path)  # A* evaluation: f(n) = g(n) + h(n)
                heappush(open_list, (priority, next_state_bytes, next_path))
                closed_set.add(next_state_bytes)

    # If the goal is unreachable
    return None, -1, len(closed_set)


In [195]:
def beam_search(state: np.ndarray, beam_size: int):
    """Solve the puzzle using Beam Search with randomized action selection."""
    priority_queue = []
    visited = set()
    total_nodes_evaluated = 0

    initial_priority = difference_from_goal(state)
    heappush(priority_queue, (initial_priority, tuple(map(tuple, state))))

    while priority_queue and not is_goal(state):
        priority, state_tuple = heappop(priority_queue)
        state = np.array(state_tuple)
        
        visited.add(state_tuple)

        if is_goal(state):
            return state, total_nodes_evaluated, len(visited)

        for _ in range(beam_size):
            new_state = do_action(state, choice(available_actions(state)))
            new_state_tuple = tuple(map(tuple, new_state))

            if new_state_tuple not in visited:
                priority = (
                    0.8 * difference_from_goal(new_state) +
                    0.2 * manhattan_distance(new_state)
                )
                heappush(priority_queue, (priority, new_state_tuple))
                total_nodes_evaluated += 1

    return state, total_nodes_evaluated, len(visited)


In [196]:
# Initialize and randomize the starting state
initial_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for _ in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    initial_state = do_action(initial_state, choice(available_actions(initial_state)))

print("Initial state:")
print(initial_state)
# Solve the puzzle
solution_path, quality, cost = A_star(initial_state)


print("A* Solution path:", solution_path)
print("Quality (number of moves):", quality)
print("Cost (total nodes evaluated):", cost)

solution_path,quality,cost = beam_search(initial_state,4)
print("Beam search Solution path:\n", solution_path)
print("Quality (number of moves):", quality)
print("Cost (total nodes evaluated):", cost)



Randomizing:   0%|          | 0/1000 [00:00<?, ?it/s]

Initial state:
[[4 1 7]
 [3 5 8]
 [6 2 0]]
A* Solution path: [Action(pos1=(2, 2), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(2, 2)), Action(pos1=(2, 2), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(0, 0)), Action(pos1=(0, 0), pos2=(0, 1)), Action(pos1=(0, 1), pos2=(0, 2)), Action(pos1=(0, 2), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(0, 1)), Action(pos1=(0, 1), pos2=(0, 0)), Action(pos1=(0, 0), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(2, 0)), Action(pos1=(2, 0), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(0, 0)), Action(pos1=(0, 0), pos2=(0, 1)), Action(pos1=(0, 1), pos2=(0, 2)), Action(pos1=(0, 2), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(2, 2)), Action(pos1=(2, 2), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(1, 2)), Action(pos1=(1, 2), 